# COVID-19 Dashboard

### Dependencies

In [1]:
!pip install plotly

In [2]:
!pip install dash

     |████████████████████████████████| 9.8 MB 12.9 MB/s 
     |████████████████████████████████| 357 kB 73.1 MB/s 


In [3]:
!pip install dash_bootstrap_components

     |████████████████████████████████| 210 kB 12.6 MB/s 


In [4]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
pio.renderers.default = "browser"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  


### Dataset

In [5]:
covid_conf_ts = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
covid_dead_ts = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
covid_recv_ts = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

### Data Processing

In [6]:
#get data in cleaned time series format for country
#returns dataframe with date and total column
#when you select a country from the dropdown, this function will prepare the time series for that country and return that as dataframe
def process_data(data,cntry='India',window=3):
    conf_ts = data
    conf_ts_cntry = conf_ts[conf_ts['Country/Region']==cntry]
    final_dataset = conf_ts_cntry.T[4:].sum(axis='columns').diff().rolling(window=window).mean()[40:]
    df = pd.DataFrame(final_dataset,columns=['Total'])
    return df
#Here data for the country selected, transposed truncating first 4 rows as they contain some latitude longitude info,
#summing all rows of a country (if there are different rows), diff function gives delta between 2 days, rolling function will help to smoothing the graph
#so this rolling window is used as the slider
#we also need to provide what it needs to do with that rolling window - mean is used so it will give an average of that window

In [7]:
#get overall wordlwide total for confirmed, recovered and dead cases
def get_overall_total(df):
    return df.iloc[:,-1].sum()

conf_overall_total = get_overall_total(covid_conf_ts)
dead_overall_total = get_overall_total(covid_dead_ts)
recv_overall_total = get_overall_total(covid_recv_ts)
print('Overall Confirmed:',conf_overall_total)
print('Overall Dead:',dead_overall_total)
print('Overall Recovered:',recv_overall_total)

Overall Confirmed: 522750289
Overall Dead: 6269315
Overall Recovered: 0


In [8]:
#get total for confirmed, recovered and dead for country (similar to overall total)
def get_cntry_total(df,cntry='India'):
    return df[df['Country/Region']==cntry].iloc[:,-1].sum()

cntry = 'India'
conf_cntry_total = get_cntry_total(covid_conf_ts,cntry)
dead_cntry_total = get_cntry_total(covid_dead_ts,cntry)
recv_cntry_total = get_cntry_total(covid_recv_ts,cntry)
print(f'{cntry} Confirmed:',conf_cntry_total)
print(f'{cntry} Dead:',dead_cntry_total)
print(f'{cntry} Recovered:',recv_cntry_total)

India Confirmed: 43127199
India Dead: 524293
India Recovered: 0


### Generate Line Graph using Plotly

In [9]:
#plotly px.line function is used to plot the line chart based on the data frame that is returned from process_data function
#also some minor layout changes have been done like centering the title,and giving the page colour and paper colour
#also naming the axis o the chart
#finally returning as a figure object which will be used by the dash app when we show it on the dash board

def fig_world_trend(cntry='India',window=3):
    df = process_data(data=covid_conf_ts,cntry=cntry,window=window)
    df.head(10)
    if window==1:
        yaxis_title = "Daily Cases"
    else:
        yaxis_title = "Daily Cases ({}-day MA)".format(window)
    fig = px.line(df, y='Total', x=df.index, title='Daily confirmed cases trend for {}'.format(cntry),height=600,color_discrete_sequence =['black'])
    fig.update_layout(title_x=0.5,plot_bgcolor='#ccffff',paper_bgcolor='#ccffff',xaxis_title="Date",yaxis_title=yaxis_title)
    return fig

### DASH app

In [10]:
external_stylesheets = [dbc.themes.BOOTSTRAP]
#general app that you need to start with
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = 'Covid-19 Dashboard'

In [11]:
colors = {
    'background': '#111111',
    'bodyColor':'#ccffff',
    'text': '#7FDBFF'
}
def get_page_heading_style():
    return {'backgroundColor': colors['background']}


def get_page_heading_title():
    return html.H1(children='COVID-19 Dashboard',
                                        style={
                                        'textAlign': 'center',
                                        'color': colors['text']
                                    })

def get_page_heading_subtitle():
    return html.Div(children='Visualize Covid-19 data generated from sources all over the world.',
                                         style={
                                             'textAlign':'center',
                                             'color':colors['text']
                                         })

def generate_page_header():
  #the main_header and subtitle_header are dbc components
  #the get_page_heading_style function in both main and subtitle is for altering background colour only
    main_header =  dbc.Row(
                            [
                                dbc.Col(get_page_heading_title(),md=12)
                            ],
                            align="center",
                            style=get_page_heading_style()
                        )
    subtitle_header = dbc.Row(
                            [
                                dbc.Col(get_page_heading_subtitle(),md=12)
                            ],
                            align="center",
                            style=get_page_heading_style()
                        )
    #header variable contains 2 dbs rows
    header = (main_header,subtitle_header)
    return header

### Select Country dropdown

In [12]:
def get_country_list():
    return covid_conf_ts['Country/Region'].unique()

def create_dropdown_list(cntry_list):
    dropdown_list = []
    for cntry in sorted(cntry_list):
      #dropdown needs {label, value} format
        tmp_dict = {'label':cntry,'value':cntry}
        dropdown_list.append(tmp_dict)
    return dropdown_list

#This html div will contain dcc and html components
def get_country_dropdown(id):
    return html.Div([
                        html.Label('Select Country'),
                        dcc.Dropdown(id='my-id'+str(id),
                            options=create_dropdown_list(get_country_list()),
                            value='India'
                        ),
                        html.Div(id='my-div'+str(id))
                    ])

In [13]:
#Putting the plotly figure in a dcc container
def graph1():
    return dcc.Graph(id='graph1',figure=fig_world_trend('India'))

In [14]:
#this function is used to create the content for a particular card. We have 3 card being shown on the dashboard - recovered, confirmed, dead
#
def generate_card_content(card_header,card_value,overall_value):
    card_head_style = {'textAlign':'center','fontSize':'150%'}
    card_body_style = {'textAlign':'center','fontSize':'200%'}
    card_header = dbc.CardHeader(card_header,style=card_head_style)
    card_body = dbc.CardBody(
        [
            html.H5(f"{int(card_value):,}", className="card-title",style=card_body_style),
            html.P(
                "Worlwide: {:,}".format(overall_value),
                className="card-text",style={'textAlign':'center'}
            ),
        ]
    )
    card = [card_header,card_body]
    return card

In [15]:
#In this function, the above function is called with 3 times - for recovered, confirmed and dead.
def generate_cards(cntry='India'):
    conf_cntry_total = get_cntry_total(covid_conf_ts,cntry)
    dead_cntry_total = get_cntry_total(covid_dead_ts,cntry)
    recv_cntry_total = get_cntry_total(covid_recv_ts,cntry)
    cards = html.Div(
        [
            dbc.Row(
                [
                    #dbc.Col(dbc.Card(generate_card_content("Recovered",recv_cntry_total,recv_overall_total), color="success", inverse=True),md=dict(size=2,offset=3)),
                    dbc.Col(dbc.Card(generate_card_content("Confirmed",conf_cntry_total,conf_overall_total), color="warning", inverse=True),md=dict(size=2, offset=4)),
                    dbc.Col(dbc.Card(generate_card_content("Dead",dead_cntry_total,dead_overall_total),color="dark", inverse=True),md=dict(size=2)),
                ],
                className="mb-4",
            ),
        ],id='card1'
    )
    return cards
#All these non-helper functions will be used later on in the app layout

In [16]:
def get_slider():
    return html.Div([  
                        dcc.Slider(
                            id='my-slider',
                            min=1,
                            max=15,
                            step=None,
                            marks={
                                1: '1',
                                3: '3',
                                5: '5',
                                7: '1-Week',
                                14: 'Fortnight'
                            },
                            value=3,
                        ),
                        html.Div([html.Label('Select Moving Average Window')],id='my-div'+str(id),style={'textAlign':'center'})
                    ])

In [17]:
#Now, we are generating the app layout. It will arrange all your components in a particular order
def generate_layout():
    page_header = generate_page_header()
    #assigning a dbc component to the layout variable
    layout = dbc.Container(
        [
            page_header[0], #page title
            page_header[1], #page subtitle
            html.Hr(), #horizontal break
            generate_cards(), #cards
            html.Hr(), #horizontal break
            dbc.Row(
                [
                    dbc.Col(get_country_dropdown(id=1),md=dict(size=4,offset=4))                    
                ]
            
            ), #for dropdown
            dbc.Row(
                [
                    dbc.Col(get_slider(),md=dict(size=4,offset=4))                    
                ]
            
            ), #for slider
            dbc.Row(
                [                
                    
                    dbc.Col(graph1(),md=dict(size=6,offset=3))
        
                ],
                align="center",

            ), #for graph
            
        ],fluid=True,style={'backgroundColor': colors['bodyColor'] #html options
        }
    )
    return layout


In [18]:
#assigning the generated layout to app.layout
app.layout = generate_layout()

In [19]:
#before we run the server, we need to assign callbacks because we have user interactable elements - dropdown and slider as inputs
#based on their values, the outputs - cards and graphs should get update
#for this, we use this decorator - @app.callback and assign all your inputs and outputs in lists

#then there is update_output_div function to which this decorator is attached and you call the respective functions
#which give the outputs that need to be changed with updated values

@app.callback(
    [Output(component_id='graph1',component_property='figure'), #line chart
    Output(component_id='card1',component_property='children')], #overall card numbers
    [Input(component_id='my-id1',component_property='value'), #dropdown
     Input(component_id='my-slider',component_property='value')] #slider
)
def update_output_div(input_value1,input_value2):
    return fig_world_trend(input_value1,input_value2),generate_cards(input_value1)

### Run server

In [ ]:
app.run_server(host= '0.0.0.0',debug=False)

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8050/ (Press CTRL+C to quit)
